In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [2]:
import os 
import tensorflow as tf
from pathlib import Path
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from src.architectures.simple.simple_base import SimpleBase
from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from src.architectures.adv.guendel19 import densenet

Using TensorFlow backend.


In [3]:
# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:42:00.0, compute capability: 7.5



In [4]:
!remote_access/get_tunnels.sh

URLs open for the following services:
[('tensorboard', 'http://f5bb2970.ngrok.io'), ('jupyternotebook', 'http://686ce90c.ngrok.io'), ('jupyterlab', 'http://f5ae6329.ngrok.io')]


In [5]:
chexpert_columns = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion',
                       'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax',
                       'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

chexpert_benchmark = Benchmark(Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                  chexpert_columns,
                                  epochs=5,
                                  train_labels="train.csv",
                                  path_column="Path",
                                  split_group='patient_id',
                                  batch_size=256,
                                  metrics=[
                                      "categorical_crossentropy",
                                      "categorical_accuracy",
                                      "categorical_hinge"])
chexpert_benchmark.as_dict()

{'dataset_name': 'chexpert_full',
 'dataset_folder': 'data/chexpert/full',
 'models_dir': 'models',
 'epochs': 5,
 'optimizer': 'Adam',
 'loss': 'categorical_crossentropy',
 'metrics': ['categorical_crossentropy',
  'categorical_accuracy',
  'categorical_hinge'],
 'label_columns': ['No Finding',
  'Enlarged Cardiomediastinum',
  'Cardiomegaly',
  'Lung Opacity',
  'Lung Lesion',
  'Edema',
  'Consolidation',
  'Pneumonia',
  'Atelectasis',
  'Pneumothorax',
  'Pleural Effusion',
  'Pleural Other',
  'Fracture',
  'Support Devices'],
 'path_column': 'Path',
 'path_column_prefix': '',
 'shuffle': True,
 'batch_size': 256,
 'dim': (256, 256),
 'n_channels': 3,
 'nan_replacement': 0,
 'unc_value': -1,
 'u_enc': 'uzeroes',
 'drop_last': False,
 'train_num_samples': 143379,
 'valid_num_samples': 35747,
 'test_num_samples': 44288}

In [6]:
chestxray14_columns = ['Edema', 'Atelectasis', 'Pneumonia',
                       'Pleural_Thickening', 'Cardiomegaly', 'Infiltration', 'Consolidation',
                       'Fibrosis', 'No Finding', 'Effusion', 'Nodule', 'Mass', 'Hernia',
                       'Emphysema', 'Pneumothorax']
chestxray14_benchmark = Benchmark(Path(os.environ.get("CHESTXRAY14_DATASET_DIRECTORY")),
                                  chestxray14_columns,
                                  epochs=5,
                                  train_labels="meta/data/labels.csv",
                                  path_column="Image Index",
                                  path_column_prefix="images/",
                                  split_group='Patient ID',
                                  batch_size=256,
                                  metrics=[
                                      "categorical_crossentropy",
                                      "categorical_accuracy",
                                      "categorical_hinge"])
chestxray14_benchmark.as_dict()

{'dataset_name': 'chestxray14_scale_256',
 'dataset_folder': 'data/chestxray14/scale_256',
 'models_dir': 'models',
 'epochs': 5,
 'optimizer': 'Adam',
 'loss': 'categorical_crossentropy',
 'metrics': ['categorical_crossentropy',
  'categorical_accuracy',
  'categorical_hinge'],
 'label_columns': ['Edema',
  'Atelectasis',
  'Pneumonia',
  'Pleural_Thickening',
  'Cardiomegaly',
  'Infiltration',
  'Consolidation',
  'Fibrosis',
  'No Finding',
  'Effusion',
  'Nodule',
  'Mass',
  'Hernia',
  'Emphysema',
  'Pneumothorax'],
 'path_column': 'Image Index',
 'path_column_prefix': 'images/',
 'shuffle': True,
 'batch_size': 256,
 'dim': (256, 256),
 'n_channels': 3,
 'nan_replacement': 0,
 'unc_value': -1,
 'u_enc': 'uzeroes',
 'drop_last': False,
 'train_num_samples': 72432,
 'valid_num_samples': 17948,
 'test_num_samples': 21740}

In [7]:
resnet_chestxray = SimpleBase(ResNet152V2, len(chestxray14_columns)).get_model()
resnet_chexpert = SimpleBase(ResNet152V2, len(chexpert_columns)).get_model()
#densenet = SimpleBase(DenseNet121, len(chestxray14_columns)).get_model()
#inceptionnet = SimpleBase(InceptionV3, len(chestxray14_columns)).get_model()

In [9]:
experiment_resnet_chexpert = Experiment(chexpert_benchmark, resnet_chexpert, "ResNet152V2_Test")
print(experiment_resnet_chexpert.model_description)

Trained ResNet152V2_Test architecture using predefined benchmark.The benchmark was initialized for the chexpert_full dataset with batch size of 256, shuffel set to True and images rescaled to dimension (256, 256).
The training was done for 5 epochs using the Adam optimizer and categorical_crossentropy loss.
A total of 14 labels/pathologies were included in the training and encoded using the 'uzeroes' method.
The traing set included 143379 number of sample, the validation set 35747, and the test set 44288. 


In [10]:
experiment_resnet_chestxray = Experiment(chestxray14_benchmark, resnet_chestxray, "ResNet152V2_Test")
print(experiment_resnet_chestxray.model_description)

Trained ResNet152V2_Test architecture using predefined benchmark.The benchmark was initialized for the chestxray14_scale_256 dataset with batch size of 256, shuffel set to True and images rescaled to dimension (256, 256).
The training was done for 5 epochs using the Adam optimizer and categorical_crossentropy loss.
A total of 15 labels/pathologies were included in the training and encoded using the 'uzeroes' method.
The traing set included 72432 number of sample, the validation set 17948, and the test set 21740. 


In [ ]:
exp_result_resnet_chexpert =  experiment_resnet_chexpert.run()

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 561 steps, validate for 140 steps
Epoch 1/5
244/561 [============>.................] - ETA: 7:47 - loss: 4.9427 - categorical_crossentropy: 4.9454 - categorical_accuracy: 0.0354 - categorical_hinge: 0.5684

In [ ]:
exp_result_resnet_chestxray =  experiment_resnet_chestxray.run()

In [36]:
testgen = resnet_chestxray.benchmark.testgen

testgen.on_epoch_end()

predictions = resnet_chestxray.model.predict(testgen, steps=len(testgen), verbose=1)


346/346 [==============================] - 91s 263ms/step


In [37]:
import numpy as np
predictions_bool = (predictions >= 0.5)

y_pred = np.array(predictions_bool, dtype=int)

groundtruth_label = testgen.get_encoded_labels()


In [38]:
print(len(y_pred))
print(len(groundtruth_label))

22137
22137


In [39]:

from sklearn.metrics import classification_report
report = classification_report(groundtruth_label, y_pred, target_names=list(resnet_chestxray.benchmark.label_columns))
print(report)

              precision    recall  f1-score   support

       Edema       0.03      0.55      0.06       432
 Atelectasis       0.11      0.82      0.19      2368
   Pneumonia       0.02      0.41      0.03       277
   Emphysema       0.02      0.00      0.01       456

   micro avg       0.07      0.65      0.13      3533
   macro avg       0.04      0.45      0.07      3533
weighted avg       0.08      0.65      0.14      3533
 samples avg       0.08      0.10      0.09      3533



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
valgen = resnet_chestxray.benchmark.valgen
STEP_SIZE_EVAL = len(valgen.index) // valgen.batch_size
print(len(valgen.index))
STEP_SIZE_EVAL // 10

27

In [13]:

eval_res = resnet_chestxray.model.evaluate(x=valgen, steps=STEP_SIZE_EVAL, verbose=1)


  ...
    to  
  ['...']
271/271 [==============================] - 75s 276ms/step - loss: 0.2603 - categorical_crossentropy: 0.2603 - categorical_accuracy: 0.2547 - categorical_hinge: 1.7244


In [22]:
[float (i) for i in eval_res]

[0.26028381849938104,
 0.26028382778167725,
 0.25472787022590637,
 1.7244476079940796]

In [19]:
metrics = ["categorical_crossentropy", "categorical_accuracy", "categorical_hinge"]
dict(zip (["loss"] + metrics, [float (i) for i in eval_res]))

{'loss': 0.26028381849938104,
 'categorical_crossentropy': 0.26028383,
 'categorical_accuracy': 0.25472787,
 'categorical_hinge': 1.7244476}

In [12]:
print(resnet_chestxray.evaluation_result["report"])

                    precision    recall  f1-score   support

             Edema       0.00      0.00      0.00       455
       Atelectasis       0.07      0.00      0.00      2207
         Pneumonia       0.00      0.00      0.00       276
         Emphysema       0.00      0.00      0.00       466
      Pneumothorax       0.00      0.00      0.00      1018
Pleural_Thickening       0.00      0.00      0.00       654
      Cardiomegaly       0.00      0.00      0.00       546
      Infiltration       0.18      1.00      0.30      3981
     Consolidation       0.00      0.00      0.00       963
          Fibrosis       0.00      0.00      0.00       333
        No Finding       0.54      1.00      0.70     12130
          Effusion       0.26      0.18      0.21      2626
            Nodule       0.00      0.00      0.00      1351
              Mass       0.00      0.00      0.00      1257
            Hernia       0.00      0.00      0.00        52

         micro avg       0.36      0.5

In [13]:
import numpy as np
from sklearn.metrics import classification_report
classification_report

<function sklearn.metrics._classification.classification_report(y_true, y_pred, *, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=False, zero_division='warn')>

In [33]:
y_pred = np.array([[1,0,0,0], [0,1,0,1], [1,1,0,1]])
y_true = np.array([[0,0,1,1], [0,1,1,0], [1,0,0,1]])
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.50      1.00      0.67         1
           2       0.00      0.00      0.00         2
           3       0.50      0.50      0.50         2

   micro avg       0.50      0.50      0.50         6
   macro avg       0.38      0.62      0.46         6
weighted avg       0.33      0.50      0.39         6
 samples avg       0.39      0.50      0.43         6



False

In [ ]:
model_name = 'güendel19'
model = densenet()
#benchmark = BenchmarkOne(model, model_name, **params)
#benchmark.fit_model()
#benchmark.save_model()

